In [6]:
import collections

import torch

from KGDataLoader import parse_args
import numpy as np
import os
import random
import time
from copy import deepcopy

import torch.nn as nn

from env.hgnn import hgnn_env
import logging 

### Trash

In [20]:
def seed(random_seed):
    torch.manual_seed(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)

def get_logger(logger_name, log_file, level=logging.INFO):
    l = logging.getLogger(logger_name)
    formatter = logging.Formatter('%(asctime)s : %(message)s', "%Y-%m-%d %H:%M:%S")
    fileHandler = logging.FileHandler(log_file, mode='a')
    fileHandler.setFormatter(formatter)

    l.setLevel(level)
    l.addHandler(fileHandler)

    return logging.getLogger(logger_name)
    
class Args:
    seed = 123
    task = 'rec'
    data_name = 'douban_movie'
    data_dir = 'data/'
    use_pretrain = 0
    pretrain_embedding_dir = 'datasets/pretrain/'
    cf_batch_size = 90000
    kg_batch_size = 10000
    nd_batch_size = 5000
    rl_batch_size = 1
    train_batch_size = 2000
    test_batch_size = 20000
    entity_dim = 64
    relation_dim = 32
    aggregation_type = 'bi-interaction'
    cf_l2loss_lambda = 1e-5
    lr = 0.01
    mpset = "[[['2', '1']], [['1', '2']]]"
    log = ''
    num_heads = [4]
    hidden_dim = 64

seed(0)

args=Args()
infor = 'rl_' + str(args.data_name) + '_' + str(args.task) + '_' + str(args.log)
model_name = 'model_' + infor + '.pth'
dataset = args.data_name
logger1 = get_logger('log', 'log/logger_' + infor + '.log')
logger2 = get_logger('log2', 'log/logger2_' + infor + '.log')

### Load Data

In [21]:
env = hgnn_env(logger1, logger2, model_name, args, dataset=dataset)

In [22]:
fr1 = open('./data/douban_movie/embedding/user.embedding_' + str(64), 'r')
fr2 = open('./data/douban_movie/embedding/item.embedding_' + str(64), 'r')

emb = env.train_data.x
emb.requires_grad = False

for line in fr1.readlines():
    embeddings = line.strip().split()
    id, embedding = int(embeddings[0]), embeddings[1:]
    embedding = list(map(float, embedding))
    emb[id] = torch.tensor(embedding)

for line in fr2.readlines():
    embeddings = line.strip().split()
    id, embedding = int(embeddings[0]), embeddings[1:]
    embedding = list(map(float, embedding))
    emb[id] = torch.tensor(embedding)

# emb.requires_grad = True
env.train_data.x = emb.to('cpu')

In [31]:
emb.shape

torch.Size([37595, 64])

In [36]:
env.train_data.nodes('n0')

tensor([    0,     1,     2,  ..., 12674, 12675, 12676])